#### Benchmarks to motivate switching to PySpice's NGSPICE interface over LTSPICE command line interface

In [101]:
import numpy as np

import PySpice
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

from os import system
import ltspice

In [124]:
def sim_ltspice(n):
    circuit = Circuit('test')

    circuit.V(1, 1, 0, f'{{V1}}')
    circuit.R(1, 2, 1, 5)
    circuit.R(2, 2, 0, 1)


    xs = np.random.randn(n)
    ys = np.empty(n)

    for i, x in enumerate(xs):
        circuit.parameter('V1', x)
        simulator = circuit.simulator()
        analysis = simulator.operating_point()
        ys[i] = analysis['2'][0]

        netlist = str(circuit)

        with open(f'{circuit.title}.cir', 'wt') as f:
            f.write(netlist + '.op\n')

        system(f'/Applications/LTspice.app/Contents/MacOS/LTspice -b {circuit.title}.cir')
        
        l = ltspice.Ltspice(f'{circuit.title}.raw')
        l.parse()
        ys[i] = l.get_data('V(2)')[0]

    return ys


def sim_loop(n):
    circuit = Circuit('test')

    circuit.V(1, 1, 0, f'{{V1}}')
    circuit.R(1, 2, 1, 5)
    circuit.R(2, 2, 0, 1)


    xs = np.random.randn(n)
    ys = np.empty(n)

    for i, x in enumerate(xs):
        circuit.parameter('V1', x)
        simulator = circuit.simulator()
        analysis = simulator.operating_point()
        ys[i] = analysis['2'][0]

    return ys

def sim_sweep(n):
    circuit = Circuit('test')

    circuit.V('dummy', 'dummy1', 0, -1)
    
    circuit.R(1, 2, 1, 5)
    circuit.R(2, 2, 0, 1)

    xs = np.random.randn(n)
    arr = [str(val) for pair in zip(range(1, len(xs)+1), xs) for val in pair]
    ss = ', '.join(arr)
    s = f'pwl(V(dummy1), {ss})'
    circuit.B(1, 1, 0, v=s)

    simulator = circuit.simulator()
    analysis = simulator.dc(Vdummy=slice(1, n, 1))

    return analysis['2']


##### Time benchmarks
Clearly, the NGSpice implementations are much faster (since they don't have to write to a file). It's much faster still to treat multiple evaluations on a single (fixed) circuit as parameter (voltage) sweeps over time and extract the results as such

In [126]:
%timeit sim_ltspice(100)

2.97 s ± 68.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit sim_loop(100)

80.7 ms ± 6.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [99]:
%timeit sim_sweep(100)

2.02 ms ± 46.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can see that all 3 methods are equivalent and yield correct results

In [125]:
sim_ltspice(100)

array([ 0.03753552,  0.07515843,  0.02449396, -0.2364496 , -0.13480079,
       -0.01133541,  0.338294  , -0.18403737, -0.29598153,  0.28983209,
        0.29039168,  0.29880744,  0.0798609 , -0.03220775, -0.14470004,
       -0.22870971, -0.12719533, -0.35808623, -0.09261359,  0.00710801,
       -0.03420829,  0.28246838, -0.14557856,  0.00908494,  0.03511909,
        0.07597739, -0.09303888,  0.0851147 ,  0.02474598,  0.14610714,
       -0.0972068 , -0.02091046,  0.27078012,  0.16743305,  0.25520605,
        0.08348022, -0.02364058, -0.13090456,  0.06406467,  0.0687949 ,
        0.16362356,  0.10467349,  0.04359397, -0.07487158,  0.22197866,
        0.06946041, -0.07002871,  0.0234651 ,  0.24606387,  0.06372093,
       -0.23425278,  0.06892099, -0.31014207, -0.05421357,  0.06740066,
        0.13865423, -0.19310217, -0.12016212, -0.14590812,  0.19561708,
        0.31243864, -0.00499533,  0.03768062,  0.00767007, -0.1543553 ,
        0.17335002,  0.34307778, -0.37501946,  0.15394862, -0.01

In [96]:
sim_loop(100)

array([ 0.07885199,  0.00596468,  0.10027783, -0.19254222,  0.04265089,
        0.15585951,  0.04825964, -0.14479955, -0.03139508,  0.06068564,
        0.26492146,  0.30977932,  0.05294426, -0.06801088,  0.01489242,
        0.13566437, -0.11193573, -0.17149723,  0.16622229,  0.1454099 ,
       -0.25769422,  0.01604261,  0.27451256, -0.01951146, -0.00546602,
       -0.20456149, -0.07378468,  0.25498283,  0.07233379,  0.19071317,
       -0.05652476,  0.00870971,  0.41615437,  0.06575635,  0.01684677,
       -0.05334982, -0.14993801, -0.27818801,  0.20542711, -0.11969818,
       -0.05676845, -0.22124776,  0.00732349,  0.27310307,  0.15492283,
       -0.15300501,  0.0947758 , -0.3269793 , -0.20635242,  0.08809042,
        0.21953174,  0.17563762, -0.13342021,  0.27200931, -0.04283765,
       -0.12808769,  0.05405765, -0.08851511, -0.172861  ,  0.26988958,
       -0.43738956, -0.14483025, -0.10971197,  0.13188113, -0.24398625,
       -0.08017224,  0.02791431,  0.22701923,  0.0020701 ,  0.17

In [97]:
sim_sweep(100)

WaveForm 2 [ 0.02441613 -0.34081    -0.1528805   0.21790592 -0.11922598  0.18016186
 -0.38575869  0.00135926  0.24443974 -0.0653691  -0.0072039  -0.12379952
 -0.07608914  0.00447689 -0.02725391  0.0897094   0.25522818  0.1879693
  0.07901359 -0.07360227  0.08300477 -0.00988042  0.0343416  -0.18854794
 -0.2059747  -0.1155127  -0.07819007 -0.11610655  0.21581297 -0.05313878
 -0.0468844   0.09001434 -0.09105134 -0.01706024  0.25172506  0.21968816
 -0.05923915  0.05104817 -0.10104417  0.02254201  0.04404495 -0.10922652
  0.06505453  0.19943546  0.01068215  0.01270706 -0.05390161 -0.14618208
  0.02327254  0.06133501  0.11363337  0.02501201 -0.10862106  0.05957903
  0.14300351  0.14322172 -0.05410551 -0.17340947 -0.13848567  0.23062845
  0.3830449  -0.18915428 -0.09034973  0.00595632 -0.0358194   0.02024012
  0.04581613  0.29006454 -0.00454236  0.21103563 -0.1897534  -0.20186578
 -0.22288207 -0.10847534  0.13227957 -0.06142107  0.05344406 -0.16642306
 -0.18742295 -0.21643933 -0.1086559   0.0